<a href="https://colab.research.google.com/github/IvaroEkel/Probabilistic-Machine-Learning_Lecture/blob/main/Regularization_OLS_L1_L2_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_diabetes, fetch_california_housing
from sklearn.linear_model import Lasso, Ridge, LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import pandas as pd


In [ ]:

# Load datasets
diabetes = load_diabetes()
X_diabetes, y_diabetes = diabetes.data, diabetes.target
feature_names = diabetes.feature_names
california = fetch_california_housing()
X_california, y_california = california.data, california.target

# Create DataFrame for better feature handling
diabetes_df = pd.DataFrame(X_diabetes, columns=feature_names)
california_df = pd.DataFrame(X_california)

# Standardize the features
scaler = StandardScaler()
X_diabetes = scaler.fit_transform(X_diabetes)
X_california = scaler.fit_transform(X_california)

# Train-test split
X_train_d, X_test_d, y_train_d, y_test_d = train_test_split(X_diabetes, y_diabetes, test_size=0.3, random_state=42)
X_train_c, X_test_c, y_train_c, y_test_c = train_test_split(X_california, y_california, test_size=0.3, random_state=42)


In [ ]:

# Function to fit models and compare
def compare_l1_l2(X_train, X_test, y_train, y_test, dataset_name):
    alphas = np.logspace(-2, 3, 40)
    lasso_mse = []
    ridge_mse = []

    for alpha in alphas:
        lasso = Lasso(alpha=alpha, max_iter=10000)
        ridge = Ridge(alpha=alpha)

        lasso.fit(X_train, y_train)
        ridge.fit(X_train, y_train)

        y_pred_lasso = lasso.predict(X_test)
        y_pred_ridge = ridge.predict(X_test)

        lasso_mse.append(mean_squared_error(y_test, y_pred_lasso))
        ridge_mse.append(mean_squared_error(y_test, y_pred_ridge))

    plt.figure(figsize=(8,6))
    plt.plot(alphas, lasso_mse, label='Lasso (L1)', marker='o')
    plt.plot(alphas, ridge_mse, label='Ridge (L2)', marker='x')
    plt.xscale('log')
    plt.xlabel('Regularization strength (alpha)')
    plt.ylabel('Mean Squared Error (MSE)')
    plt.title(f'Comparison of L1 and L2 Regularization on {dataset_name} Dataset')
    plt.legend()
    plt.grid(True)
    plt.show()

    print(f"\nBest Lasso (L1) MSE for {dataset_name}: {np.min(lasso_mse):.3f} at alpha={alphas[np.argmin(lasso_mse)]:.3f}")
    print(f"Best Ridge (L2) MSE for {dataset_name}: {np.min(ridge_mse):.3f} at alpha={alphas[np.argmin(ridge_mse)]:.3f}")

# Run comparison
compare_l1_l2(X_train_d, X_test_d, y_train_d, y_test_d, "Diabetes")


In [ ]:

# Extra: visualize coefficients shrinkage
alpha = 1.0
lasso = Lasso(alpha=alpha, max_iter=10000)
ridge = Ridge(alpha=alpha)
linear = LinearRegression()

lasso.fit(X_train_d, y_train_d)
ridge.fit(X_train_d, y_train_d)
linear.fit(X_train_d, y_train_d)

plt.figure(figsize=(10,6))
plt.plot(lasso.coef_, label='Lasso coefficients', marker='o')
plt.plot(ridge.coef_, label='Ridge coefficients', marker='x')
plt.plot(linear.coef_, label='Linear Regression coefficients', marker='^')
plt.xlabel('Feature Index')
plt.ylabel('Coefficient Value')
plt.axhline(0, color='black', linewidth=0.8)
plt.title('Coefficient Comparison (Diabetes dataset)')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:

# Visualization of level surfaces and coefficient paths
alphas = np.logspace(-2, 2, 30)
coeffs_lasso = []
coeffs_ridge = []

for alpha in alphas:
    lasso = Lasso(alpha=alpha, max_iter=10000)
    ridge = Ridge(alpha=alpha)
    lasso.fit(X_train_d[:, :2], y_train_d)  # Using only the first two features: feature_names[0] and feature_names[1]
    ridge.fit(X_train_d[:, :2], y_train_d)
    coeffs_lasso.append(lasso.coef_)
    coeffs_ridge.append(ridge.coef_)

coeffs_lasso = np.array(coeffs_lasso)
coeffs_ridge = np.array(coeffs_ridge)

df_coeffs_lasso = pd.DataFrame(coeffs_lasso, columns=feature_names[:2])
df_coeffs_ridge = pd.DataFrame(coeffs_ridge, columns=feature_names[:2])

fig, axs = plt.subplots(1, 2, figsize=(12, 5))
x = np.linspace(-1, 1, 100)
y = np.linspace(-1, 1, 100)
X_grid, Y_grid = np.meshgrid(x, y)

# L1 norm contour
Z1 = np.abs(X_grid) + np.abs(Y_grid)
axs[0].contour(X_grid, Y_grid, Z1, levels=[0.5, 1, 1.5], colors='blue')
axs[0].plot(coeffs_lasso[:, 0], coeffs_lasso[:, 1], marker='o', label='Lasso path', color='black')
axs[0].set_title(f'L1-norm constraint and Coefficient Path (Lasso)\n({feature_names[0]} vs {feature_names[1]})')
axs[0].set_xlabel(f'Coefficient for {feature_names[0]}')
axs[0].set_ylabel(f'Coefficient for {feature_names[1]}')
axs[0].grid(True)
axs[0].axhline(0, color='black', lw=0.5)
axs[0].axvline(0, color='black', lw=0.5)
axs[0].set_aspect('equal')
axs[0].legend()

# L2 norm contour
Z2 = np.sqrt(X_grid**2 + Y_grid**2)
axs[1].contour(X_grid, Y_grid, Z2, levels=[0.5, 1, 1.5], colors='red')
axs[1].plot(coeffs_ridge[:, 0], coeffs_ridge[:, 1], marker='x', label='Ridge path', color='black')
axs[1].set_title(f'L2-norm constraint and Coefficient Path (Ridge)\n({feature_names[0]} vs {feature_names[1]})')
axs[1].set_xlabel(f'Coefficient for {feature_names[0]}')
axs[1].set_ylabel(f'Coefficient for {feature_names[1]}')
axs[1].grid(True)
axs[1].axhline(0, color='black', lw=0.5)
axs[1].axvline(0, color='black', lw=0.5)
axs[1].set_aspect('equal')
axs[1].legend()

plt.tight_layout()
plt.show()



In [ ]:
df_coeffs_lasso

In [ ]:
# Explicitly fit models for 5 alphas
selected_alphas = [0.01, 0.1, 1, 10, 100]

for alpha in selected_alphas:
    print(f"\nAlpha = {alpha}")
    lasso = Lasso(alpha=alpha, max_iter=10000)
    ridge = Ridge(alpha=alpha)
    linear = LinearRegression()

    lasso.fit(X_train_d, y_train_d)
    ridge.fit(X_train_d, y_train_d)
    linear.fit(X_train_d, y_train_d)

    df = pd.DataFrame({
        'Feature': feature_names,
        'OLS': linear.coef_,
        f'Lasso (alpha={alpha})': lasso.coef_,
        f'Ridge (alpha={alpha})': ridge.coef_
    })

    print(df.round(3))
